In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import scipy.stats as stats
from datetime import datetime as dt

from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer

from sklearn.model_selection import train_test_split

import statsmodels
import statsmodels.api as sm

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, cohen_kappa_score, confusion_matrix, roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, f1_score

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import tree

from sklearn.feature_selection import RFE

from warnings import filterwarnings
filterwarnings('ignore')


import pandas as pd

# ---------------------------------------------------------------------
# 1. LOAD DATA DASAR
# ---------------------------------------------------------------------
orders = pd.read_csv('olist_orders_dataset.csv', parse_dates=['order_purchase_timestamp'])
customers = pd.read_csv('olist_customers_dataset.csv')
order_items = pd.read_csv('olist_order_items_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')

# (Opsional) Jika Anda ingin memanfaatkan nama kategori dalam bahasa Inggris,
# Anda bisa menggabungkan product_category_name_translation.csv

# ---------------------------------------------------------------------
# 2. GABUNGKAN DATA UNTUK MENDAPATKAN product_category_name
# ---------------------------------------------------------------------
df_merge = pd.merge(order_items, products, on='product_id', how='left')

# Gabungkan dengan orders agar punya kolom order_purchase_timestamp
df_merge = pd.merge(
    df_merge,
    orders[['order_id','customer_id','order_purchase_timestamp']],
    on='order_id',
    how='left'
)

# Gabungkan dengan customers agar punya customer_unique_id
df_merge = pd.merge(
    df_merge,
    customers[['customer_id','customer_unique_id']],
    on='customer_id',
    how='left'
)

# df_merge sekarang memuat:
# - customer_unique_id
# - product_category_name
# - order_purchase_timestamp
# - dsb.

# ---------------------------------------------------------------------
# 3. TENTUKAN FIRST_ORDER_DATE, LAST_ORDER_DATE, ORDER_COUNT PER customer_unique_id
# ---------------------------------------------------------------------
agg_data = df_merge.groupby('customer_unique_id').agg({
    'order_id': 'nunique',  # jumlah pesanan unik
    'order_purchase_timestamp': ['min','max']  # tanggal order pertama & terakhir
}).reset_index()

agg_data.columns = ['customer_unique_id','order_count','first_order_date','last_order_date']

# Hitung recency & days_since_first (seperti di kode Anda)
max_date = df_merge['order_purchase_timestamp'].max()
agg_data['recency'] = (max_date - agg_data['last_order_date']).dt.days
agg_data['days_since_first'] = (max_date - agg_data['first_order_date']).dt.days

# Buyer type
agg_data['buyer_type'] = agg_data['order_count'].apply(lambda x: 'one_time' if x == 1 else 'repeat')

# ---------------------------------------------------------------------
# 4. TENTUKAN "KATEGORI ACUAN" UNTUK SETIAP PELANGGAN
#    (Di sini kita ambil "kategori terakhir" yang dibeli)
# ---------------------------------------------------------------------
# Urutkan df_merge agar bisa rank descending (transaksi terbaru)
df_merge = df_merge.sort_values(['customer_unique_id','order_purchase_timestamp'], ascending=[True, False])

# Buat rank: 1 = transaksi paling baru
df_merge['rank_desc'] = df_merge.groupby('customer_unique_id')['order_purchase_timestamp'].rank(method='first', ascending=False)

# Ambil baris rank_desc == 1 => last order per customer
last_orders = df_merge[df_merge['rank_desc'] == 1].copy()

# Kita hanya butuh kolom: customer_unique_id & product_category_name (terakhir)
last_orders = last_orders[['customer_unique_id','product_category_name']]

# Merge last category ke agg_data
agg_data = pd.merge(agg_data, last_orders, on='customer_unique_id', how='left')

# ---------------------------------------------------------------------
# 5. HITUNG MEDIAN RECENCY & MEDIAN DAYS_SINCE_FIRST PER KATEGORI
#    (Pisahkan repeat vs one_time)
# ---------------------------------------------------------------------
# a) Repeat Buyer -> Median Recency per Category
df_repeat = agg_data[agg_data['buyer_type'] == 'repeat']

median_recency_per_cat = df_repeat.groupby('product_category_name')['recency'].median().reset_index()
median_recency_per_cat.columns = ['product_category_name','median_recency']

# b) One-Time Buyer -> Median days_since_first per Category
df_one_time = agg_data[agg_data['buyer_type'] == 'one_time']

median_first_per_cat = df_one_time.groupby('product_category_name')['days_since_first'].median().reset_index()
median_first_per_cat.columns = ['product_category_name','median_days_since_first']

# Contoh: definisikan threshold = 2 x median
median_recency_per_cat['threshold_repeat'] = median_recency_per_cat['median_recency'] * 2
median_first_per_cat['threshold_one_time'] = median_first_per_cat['median_days_since_first'] * 2

# ---------------------------------------------------------------------
# 6. MERGE THRESHOLD KEMBALI KE agg_data
# ---------------------------------------------------------------------
agg_data = pd.merge(
    agg_data,
    median_recency_per_cat[['product_category_name','threshold_repeat']],
    on='product_category_name',
    how='left'
)

agg_data = pd.merge(
    agg_data,
    median_first_per_cat[['product_category_name','threshold_one_time']],
    on='product_category_name',
    how='left'
)

# Sekarang di agg_data ada:
# - threshold_repeat (khusus repeat)
# - threshold_one_time (khusus one_time)

# ---------------------------------------------------------------------
# 7. DEFINE CHURN
# ---------------------------------------------------------------------
def define_churn(row):
    if row['buyer_type'] == 'one_time':
        # Jika threshold_one_time NaN (misal kategori tidak ada di median_first_per_cat),
        # bisa fallback ke default, misal 180
        thr = row['threshold_one_time'] if pd.notna(row['threshold_one_time']) else 180
        return 1 if row['days_since_first'] > thr else 0
    else:
        thr = row['threshold_repeat'] if pd.notna(row['threshold_repeat']) else 180
        return 1 if row['recency'] > thr else 0

agg_data['churn'] = agg_data.apply(define_churn, axis=1)

# ---------------------------------------------------------------------
# 8. ANALISIS HASIL
# ---------------------------------------------------------------------
churn_rate_total = agg_data['churn'].mean() * 100
print(f"Churn Rate (Total): {churn_rate_total:.2f}%")

# Pisahkan churn rate one-time vs repeat
one_time_data = agg_data[agg_data['buyer_type'] == 'one_time']
repeat_data   = agg_data[agg_data['buyer_type'] == 'repeat']

churn_rate_one_time = one_time_data['churn'].mean() * 100
churn_rate_repeat   = repeat_data['churn'].mean() * 100

print(f"Churn Rate (One-Time) : {churn_rate_one_time:.2f}%")
print(f"Churn Rate (Repeat)   : {churn_rate_repeat:.2f}%")

# Lihat beberapa baris hasil
print(agg_data.head(10))


Churn Rate (Total): 13.12%
Churn Rate (One-Time) : 13.14%
Churn Rate (Repeat)   : 12.46%
                 customer_unique_id  order_count    first_order_date  \
0  0000366f3b9a7992bf8c76cfdf3221e2            1 2018-05-10 10:56:27   
1  0000b849f77a49e4a4ce2b2a4ca5be3f            1 2018-05-07 11:11:27   
2  0000f46a3911fa3c0805444483337064            1 2017-03-10 21:05:03   
3  0000f6ccb0745a6a4b88665a16c9f078            1 2017-10-12 20:29:41   
4  0004aac84e0df4da2b147fca70cf8255            1 2017-11-14 19:45:42   
5  0004bd2a26a76fe21f786e4fbd80607f            1 2018-04-05 19:33:16   
6  00050ab1314c0e55a6ca13cf7181fecf            1 2018-04-20 12:57:23   
7  00053a61a98854899e70ed204dd4bafe            1 2018-02-28 11:15:41   
8  0005e1862207bf6ccc02e4228effd9a0            1 2017-03-04 23:32:12   
9  0005ef4cd20d2893f0d9fbd94d3c0d97            1 2018-03-12 15:22:12   

      last_order_date  recency  days_since_first buyer_type  \
0 2018-05-10 10:56:27      115               115   one_

In [ ]:
asdasdsa